In [12]:
import sys
if '..' not in sys.path: sys.path.append('..')

In [1]:
from datasets import load_dataset

imdb = load_dataset('stanfordnlp/imdb')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [16]:
train = imdb['train']['text']

from noise import noise_forward

len(noise_forward(train[0])[0])

33